In [1]:
import numpy as np

data_path = 'data/nonLinear_data.npy'
data = np.load(data_path, allow_pickle=True).item()
X, y = data['X'], data['labels']


In [2]:
print(X)

[[ 0.00000000e+00  0.00000000e+00]
 [ 9.60083739e-04  1.00552794e-02]
 [ 1.04586376e-02  1.72840539e-02]
 [ 8.79222065e-04  3.02902726e-02]
 [ 9.91726687e-03  3.91680265e-02]
 [ 6.77282958e-03  5.00488652e-02]
 [ 1.41908220e-02  5.89212623e-02]
 [ 3.11579559e-02  6.34718176e-02]
 [-8.03243759e-03  8.04078719e-02]
 [ 4.44744542e-02  7.92873618e-02]
 [ 4.42749312e-02  9.07897074e-02]
 [ 1.35659734e-02  1.10279841e-01]
 [ 4.51006429e-02  1.12509157e-01]
 [ 8.18222344e-02  1.02704724e-01]
 [ 1.02915603e-01  9.69862772e-02]
 [ 1.09463907e-01  1.04759220e-01]
 [ 9.88552373e-02  1.27857052e-01]
 [ 1.33913469e-01  1.07489394e-01]
 [ 1.67286205e-01  7.12262365e-02]
 [ 1.70560114e-01  8.79899069e-02]
 [ 1.40953043e-01  1.44721808e-01]
 [ 1.28611110e-01  1.68684887e-01]
 [ 1.42427503e-01  1.70578786e-01]
 [ 1.90889800e-01  1.32420423e-01]
 [ 1.87648899e-01  1.53484214e-01]
 [ 1.93996423e-01  1.61661347e-01]
 [ 2.35108650e-01  1.17031947e-01]
 [ 2.38113100e-01  1.32974874e-01]
 [ 2.06828734e-01  1

In [3]:
print (X.shape , y. shape )

(300, 2) (300,)


In [4]:
split_ratio = 0.8
random_state = 42
np.random.seed(random_state)
indices = np.random.permutation(len(X))
split_index = int(len(X) * split_ratio)
train_indices, test_indices = indices[:split_index], indices[split_index:]

X_train, X_test = X[train_indices], X[test_indices]
y_train, y_test = y[train_indices], y[test_indices]

In [5]:
print("First 5 rows of X_train:")
print(X_train[:5])
print("First 5 rows of y_train:")
print(y_train[:5])
print("First 5 rows of X_test:")
print(X_test[:5])
print("First 5 rows of y_test:")
print(y_test[:5])

First 5 rows of X_train:
[[ 0.02995783 -0.00456091]
 [-0.59477177 -0.30114945]
 [ 0.04739604  0.52310977]
 [ 0.04447445  0.07928736]
 [ 0.13882627 -0.30304847]]
First 5 rows of y_train:
[2 2 1 0 2]
First 5 rows of X_test:
[[-0.73735891 -0.00467537]
 [ 0.00942238 -0.00363995]
 [ 0.17036411  0.5921412 ]
 [ 0.4165839  -0.12293116]
 [ 0.10181868 -0.13827416]]
First 5 rows of y_test:
[2 2 1 0 2]


In [6]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.01):
        """
        Initialize the neural network with random weights and biases.
        :param input_size: Number of input features
        :param hidden_size: Number of neurons in the hidden layer
        :param output_size: Number of output neurons (equal to the number of classes)
        :param learning_rate: Learning rate for weight updates
        """
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.learning_rate = learning_rate
        
        # Initialize weights and biases
        self.W1 = np.random.randn(self.input_size, self.hidden_size)
        self.b1 = np.zeros((1, self.hidden_size))
        self.W2 = np.random.randn(self.hidden_size, self.output_size)
        self.b2 = np.zeros((1, self.output_size))
    
    def relu(self, x):
        """ReLU activation function."""
        return np.maximum(0, x)
    
    def relu_derivative(self, x):
        """Derivative of ReLU activation function."""
        return (x > 0).astype(float)
    
    def softmax(self, x):
        """Softmax activation function for multi-class classification."""
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))  # Stability improvement
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)
    
    def forward(self, X):
        """
        Perform forward propagation to compute network output.
        :param X: Input data
        :return: Softmax activated output values
        """
        self.z1 = np.dot(X, self.W1) + self.b1  # Compute input to hidden layer
        self.a1 = self.relu(self.z1)  # Use ReLU activation for hidden layer
        self.z2 = np.dot(self.a1, self.W2) + self.b2  # Compute input to output layer
        return self.softmax(self.z2)  # Use softmax activation for output layer
    
    def backward(self, X, y):
        """
        Perform backpropagation to update weights and biases.
        :param X: Input data
        :param y: True labels (one-hot encoded)
        """
        m = X.shape[0]  # Number of training examples
        
        # Compute output error
        output = self.forward(X)
        error = output - y
        dW2 = np.dot(self.a1.T, error) / m
        db2 = np.sum(error, axis=0, keepdims=True) / m
        
        # Compute hidden layer error
        hidden_error = np.dot(error, self.W2.T) * self.relu_derivative(self.z1)
        dW1 = np.dot(X.T, hidden_error) / m
        db1 = np.sum(hidden_error, axis=0, keepdims=True) / m
        
        # Update weights and biases using gradient descent
        self.W2 -= self.learning_rate * dW2
        self.b2 -= self.learning_rate * db2
        self.W1 -= self.learning_rate * dW1
        self.b1 -= self.learning_rate * db1
    
    def train(self, X, y, epochs=100):
        """
        Train the neural network using forward and backward propagation.
        :param X: Training input data
        :param y: Training labels (one-hot encoded)
        :param epochs: Number of iterations
        """
        for epoch in range(epochs):
            output = self.forward(X)
            self.backward(X, y)
            if epoch % 10 == 0:
                loss = -np.mean(np.sum(y * np.log(output + 1e-8), axis=1))  # Compute categorical cross-entropy loss
                print(f'Epoch {epoch}, Loss: {loss}')
    
    def predict(self, X):
        """
        Make predictions using the trained neural network.
        :param X: Input data
        :return: Predicted class labels
        """
        output = self.forward(X)
        return np.argmax(output, axis=1)

# Convert labels to one-hot encoding
y_one_hot = np.eye(len(set(y.flatten())))[y.flatten()]

# Initialize and train the neural network on training data
nn = NeuralNetwork(input_size=X.shape[1], hidden_size=4, output_size=y_one_hot.shape[1], learning_rate=0.1)
nn.train(X_train, y_one_hot[train_indices], epochs=100)

# Make predictions on test data
y_pred = nn.predict(X_test)
print('Predictions on test set:', y_pred)


Epoch 0, Loss: 1.102306372552096
Epoch 10, Loss: 1.026217565913595
Epoch 20, Loss: 0.9860862275493818
Epoch 30, Loss: 0.95684192681065
Epoch 40, Loss: 0.934072065316263
Epoch 50, Loss: 0.9150400171594768
Epoch 60, Loss: 0.8983851146875544
Epoch 70, Loss: 0.8838016654112099
Epoch 80, Loss: 0.8709272307446966
Epoch 90, Loss: 0.8593322075854241
Predictions on test set: [2 2 1 0 2 1 0 2 2 2 2 0 1 2 1 2 0 0 2 1 2 0 2 1 2 1 1 1 1 0 0 0 1 1 1 1 1
 0 0 2 2 1 1 2 1 2 1 2 1 2 2 2 2 1 1 0 0 2 2 2]
